In [4]:
import numpy as np
import math

def read_tsp_file(file_path):
    city_coordinates = {}
    with open(file_path, 'r') as file:
        # Temukan awal dari NODE_COORD_SECTION
        for line in file:
            if line.strip() == "NODE_COORD_SECTION":
                break
        
        # Baca data koordinat kota
        for line in file:
            if line.strip() == "EOF":
                break
            # Split line and convert values to integers
            values = list(map(int, line.split()))
            
            if len(values) == 2:
                # If there are two values, assume (x, y) and use the line number as city_id
                city_id = len(city_coordinates) + 1
                x, y = values
            elif len(values) == 3:
                # If there are three values, assume (city_id, x, y)
                city_id, x, y = values
            else:
                # Handle other cases as needed
                continue
            
            city_coordinates[city_id] = (x, y)
    
    return city_coordinates

file_path = 'D:/att48.tsp'
city_coordinates = read_tsp_file(file_path)

# Function to calculate Euclidean distance
def euclidean_distance(city1, city2):
    return math.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

# Generate initial population
def generate_initial_population(city_coordinates, population_size):
    city_ids = list(city_coordinates.keys())
    population = [np.random.permutation(city_ids).tolist() for _ in range(population_size)]
    return population

# Calculate total distance of a path
def calculate_total_distance(path, city_coordinates):
    total_distance = 0
    for i in range(len(path)):
        from_city = city_coordinates[path[i]]
        to_city = city_coordinates[path[(i + 1) % len(path)]]  # Loop back to start
        total_distance += euclidean_distance(from_city, to_city)
    return total_distance

# Fitness evaluation
def evaluate_fitness(population, city_coordinates):
    fitness_scores = []
    for individual in population:
        distance = calculate_total_distance(individual, city_coordinates)
        fitness_score = 1 / distance  # Higher fitness for shorter path
        fitness_scores.append(fitness_score)
    return np.array(fitness_scores)

# Selection - Roulette Wheel Selection
def select_parents(population, fitness_scores, num_parents):
    parents = []
    for _ in range(num_parents):
        parent_idx = np.random.choice(np.arange(len(population)), p=fitness_scores/fitness_scores.sum())
        parents.append(population[parent_idx])
    return parents

# # Crossover - Ordered Crossover
# def ordered_crossover(parent1, parent2):
#     size = len(parent1)
#     start, end = sorted(np.random.sample(2) * size)
#     child = [None]*size
#     child[int(start):int(end)] = parent1[int(start):int(end)]
#     fill_values = [item for item in parent2 if item not in child]
#     child = [item if item is not None else fill_values.pop(0) for item in child]
#     return child
def two_point_crossover(parent1, parent2):
    size = len(parent1)
    # Hasilkan dua titik berbeda untuk saling bersilangan
    crossover_points = sorted(np.random.choice(range(1, size), 2, replace=False))
    start, end = crossover_points
    
    # Initialize the child with None
    """
    menginisialisasi daftar bernama child dengan panjang sama dengan size, dimana setiap elemen dalam daftar awalnya disetel ke None.
    Menginisialisasi [None] dapat membantu memeriksa error. 
    Setelah operasi crossover selesai, jika ada nilai None yang tersisa, hal ini menunjukkan bahwa algoritma 
    tidak mengisi array child dengan benar, yang dapat membantu dalam men-debug proses crossover.
    """
    child = [None] * size
    
    # Salin segmen dari parent1 ke anak
    child[start:end] = parent1[start:end]
    # Isi sisa rute dari induk2, pastikan tidak ada duplikat
    current_position = end
    for city in parent2:
        if city not in child:
            if current_position >= size:
                current_position = 0
            child[current_position] = city
            current_position += 1
    
    return child


# Mutation - Swap Mutation
def swap_mutation(individual, mutation_rate):
    if np.random.rand() < mutation_rate:
        idx1, idx2 = np.random.choice(len(individual), 2, replace=False)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual

# Jalankan algoritma genetika untuk TSP
def genetic_algorithm_tsp_with_elitism(city_coordinates, population_size=100, num_generations=100, mutation_rate=0.01, elitism_size=2):
    population = generate_initial_population(city_coordinates, population_size)
    for generation in range(num_generations):
        fitness_scores = evaluate_fitness(population, city_coordinates)
        total_fitness = np.sum(fitness_scores)
        best_distance = 1 / np.max(fitness_scores)
        
        # Elitisme: Cadangan individu terbaik dari populasi saat ini
        elite_indices = np.argsort(-fitness_scores)[:elitism_size]
        elites = [population[i] for i in elite_indices]
        
        # Hasilkan populasi berikutnya, tidak termasuk elit untuk saat ini
        next_population = []
        parents = select_parents(population, fitness_scores, population_size - elitism_size)
        for i in range(0, len(parents), 2):
            if i+1 < len(parents):  # Ensure there's a pair for crossover
                for child in [two_point_crossover(parents[i], parents[i+1]), two_point_crossover(parents[i+1], parents[i])]:
                    child = swap_mutation(child, mutation_rate)
                    next_population.append(child)
        
        # Tambahkan elit ke populasi berikutnya setelah mutasi
        next_population.extend(elites)
        
        population = next_population
        
        # Perbarui skor kebugaran berdasarkan populasi baru termasuk elit
        fitness_scores = evaluate_fitness(population, city_coordinates)
        best_distance = 1 / np.max(fitness_scores)  # Hitung ulang jarak terbaik dengan menyertakan elit
        
        # Display generation info
        print(f"Generasi {generation + 1}\nBest Distance = {best_distance}\nTotal Fitness = {total_fitness}\n")

    # Identifikasi solusi terbaik dalam populasi akhir
    best_idx = np.argmax(fitness_scores)
    best_solution = population[best_idx]
    return best_solution, best_distance

# Jalankan Algoritma Genetika dengan elitisme dan tampilkan hasilnya
best_path, best_distance = genetic_algorithm_tsp_with_elitism(city_coordinates)
print(f"\nBest Path:{best_path}\nBest Distance: {best_distance}")

Generasi 1
Best Distance = 133700.8441819033
Total Fitness = 0.0006364633923371852

Generasi 2
Best Distance = 133700.8441819033
Total Fitness = 0.0006373637097426342

Generasi 3
Best Distance = 131665.04048834386
Total Fitness = 0.0006428417882927499

Generasi 4
Best Distance = 130955.62864389441
Total Fitness = 0.0006403633565670425

Generasi 5
Best Distance = 127419.49480895796
Total Fitness = 0.0006486339215580272

Generasi 6
Best Distance = 117365.12972875363
Total Fitness = 0.0006607135252427265

Generasi 7
Best Distance = 117365.12972875363
Total Fitness = 0.000673717600250121

Generasi 8
Best Distance = 117365.12972875363
Total Fitness = 0.0006849368664655007

Generasi 9
Best Distance = 117365.12972875363
Total Fitness = 0.0006794787493707044

Generasi 10
Best Distance = 117365.12972875363
Total Fitness = 0.0006642435279064992

Generasi 11
Best Distance = 117365.12972875363
Total Fitness = 0.000669576162591479

Generasi 12
Best Distance = 114994.10981585554
Total Fitness = 0.00

Generasi 99
Best Distance = 96061.05860289598
Total Fitness = 0.0007023567563076994

Generasi 100
Best Distance = 96061.05860289598
Total Fitness = 0.0007219444296231888


Best Path:[7, 28, 40, 9, 23, 1, 38, 33, 11, 34, 10, 41, 42, 13, 47, 35, 24, 45, 4, 26, 48, 36, 2, 25, 32, 8, 39, 5, 29, 22, 21, 20, 6, 17, 44, 31, 15, 12, 46, 18, 43, 37, 16, 3, 14, 19, 30, 27]
Best Distance: 96061.05860289598
